# Example of DC-JS library usage [cite](#cite-dcjs-notebooks)

In [1]:
from IPython.display import display, Javascript, HTML

import pandas as pd
import jinja2

In [2]:
def inline_dc(stuff):
    """
    Embeds the HTML source of the dc charts directly into the IPython notebook.
    
    This method will not work if the dc charts depends on any files(json data). 
    Also this uses the HTML5 srcdoc attribute, which may not be supported in 
    all browsers.
    """

    return HTML('''
        <iframe srcdoc="{srcdoc}" 
        style="width: 100%; height: 300px; border: none"></iframe>'
    '''.format(srcdoc=stuff.replace('"', '&quot;')))

def import_js(path_js):
    with open(path_js) as f:
        return Javascript(f.read())

In [3]:
# import js libs
head = HTML('''
<head> 
<meta http-equiv="content-type" content="text/html; charset=UTF8">

<link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/dc/2.0.0-alpha.5/dc.css"/> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.4.13/d3.min.js"></script> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/crossfilter/1.3.12/crossfilter.min.js"></script> 

<script>
{import_js}
</script>

<style>

.chart {
    margin-left:90px;
}
.table-hover tbody tr:hover td, .table-hover tbody tr:hover th {
    background-color: #f5f5f5;
}
.table th, .table td {
    padding: 8px;
    line-height: 20px;
    text-align: left;
    vertical-align: top;
    border-top: 1px solid #ddd;
}
user agent stylesheettd, th {
    display: table-cell;
    vertical-align: inherit;
}

</style>

</head>
'''.replace(
    '{import_js}', import_js('static/libs/dc/dc.min.js').data
))

In [4]:
# body
body_html = HTML("""
<!-- A div anchor that can be identified by id -->
<div id="rowchart" class="dc-chart">
    <!-- Title or anything you want to add above the chart -->
    <strong>Row Chart</strong>
    <!--
        This will create a reset button when ever the user selects a filter.
        Clicking on the reset button will remove all filters.
     -->
    <a class="reset" href="javascript:void(0);" onclick="javascript:RowChart.filterAll();dc.redrawAll();" style="display: none;">reset</a>
    <div class="clearfix"></div>
    <!--
        dc.js will also automatically inject applied current filter value into
        any html element with css class set to "filter"
    -->
    <span class="reset" style="display: none;">Current filter: <span class="filter"></span></span>
    <br/>
    <br/>
</div>

<table id="data-table" class='table'>
 <!-- create a custom header -->
    <thead class="header">
        <tr class="header">
        <th>State</th>
        <th>Population</th>
        <th>Age</th>
        <th>State2</th>
        </tr>
    </thead>
</table>
""")

In [5]:
# get data
df = pd.DataFrame({
    'population':[234,345,345,34],
    'age':[2,3,6,9],
    'state':['a','b','c','d'],
    'state2':['d','c','b','a']
})

df

,age,population,state,state2
0,2,234,a,d
1,3,345,b,c
2,6,345,c,b
3,9,34,d,a


In [14]:
# js template

dc_template = jinja2.Template(
"""   
console.clear();

// Create Global Variables
var RowChart = dc.rowChart("#rowchart");
var DataTable = dc.dataTable("#data-table");

// Load data
var dataset = {{ data }};

// Call function
Graph(dataset);

var flu_colors = {
    'a': '#dfdfdf',
    'b': '#ffcc00',
    'c': '#ff0000',
    'd': '#00ff00',
};

// Create function
function Graph(data) {
   
    // Feed it through crossfilter  
    var ndx = crossfilter(data);
    
    // for testing
    //console.log(data);

    //define a dimension
    //Here we will group by state
    var stateDim = ndx.dimension(dc.pluck('state'));

    //Here we group by state and sum on column population
    var g = stateDim.group().reduceSum(dc.pluck('population'));
    
    //Lets create a row chart
    RowChart.dimension(stateDim)
        .group(g)
        .width(500);
        
    RowChart.getColor = function(d, i){
        var flu_colors = {
            'a': '#dfdfdf',
            'b': '#ffcc00',
            'c': '#ff0000',
            'd': '#00ff00',
        };
        return flu_colors[d.key];
    };

    
    DataTable.dimension(stateDim)
        // data table does not use crossfilter group but rather a closure
        // as a grouping function
        .group(function(d) {
            return '';
        })
        .columns([
            dc.pluck('state'),
            dc.pluck('population'),
            dc.pluck('age'),
            dc.pluck('state2')
        ])
        .size(100);
    
    dc.renderAll(); // render all charts on the page
    
}; // end graph function    
    
""")

In [15]:
# bind date to js template
body_js = Javascript(dc_template.render(
    data=df.to_json(orient='records')
))

# push data to notebook

inline_dc(
    head.data + body_html.data + 
    "<script>" + body_js.data + "</script>"
)

State,Population,Age,State2
